In [46]:
function quadrant_travelcost(m)
    # cost: cheapest cost from top left corner to every other square restricted to only South/East moves. (quadrant 4)
    # and
    # dir: correct FIRST direction to go in order to take that cheapest path
    Y=size(m,1)
    X=size(m,2)
    dir=ones(Int,(Y,X))
    cost=zeros(Int,(Y,X))
    for x=1:X,y=1:Y
        if x>1
            cost[y,x]=cost[y,x-1]+m[y,x-1]
            dir[y,x]  = y==1 ? 1 : dir[y,x-1]
        end
        if y>1
            t_cost=cost[y-1,x]+m[y-1,x]
            if t_cost<cost[y,x] || x==1
                cost[y,x]=t_cost
                dir[y,x] = x==1 ? 0 : dir[y-1,x]
            end
        end
    end
    dir[1,1]=4
    return cost, dir
end

q1(m) = reverse(circshift(m, (-1,0)), dims=1)
q2(m) = q1(q3(m))
q3(m) = reverse(circshift(m, (0,-1)), dims=2)
q4(m) = m

iq1(m,d) = q1(m), replace(d, 0 => 2)
iq2(m,d) = q2(m), replace(d, 0 => 2, 1 => 3)
iq3(m,d) = q3(m), replace(d, 1 => 3)
iq4(m,d) = q4(m), d

shiftorigin(m, origin) = circshift(m, (1-origin[1], 1-origin[2]))
ishiftorigin(m, origin) = circshift(m, (origin[1]-1, origin[2]-1))

function travelcost(m, origin)
    #Cheapest cost from some point to All Other points,
    #and what direction to go on First step to take that cheapest path.
    #south/east/north/west (in that order 0,1,2,3)
    m = shiftorigin(m, origin)*0.1
    expand(f) = x -> f(x...)
    c1,d1 = m |> q1 |> quadrant_travelcost |> expand(iq1)
    c2,d2 = m |> q2 |> quadrant_travelcost |> expand(iq2)
    c3,d3 = m |> q3 |> quadrant_travelcost |> expand(iq3)
    c4,d4 = m |> q4 |> quadrant_travelcost |> expand(iq4)
    
    D = cat(d1,d2,d3,d4, dims=3)
    C = cat(c1,c2,c3,c4, dims=3)
    v, i = findmin(C, dims=3)
    
    cost = ishiftorigin(C[i][:,:,1], origin)
    first_direction = ishiftorigin(D[i][:,:,1], origin)
    return cost, first_direction
end

function score(M, ship, shipyard, threshold)
    #reward
    #minus
    #cost (from ship, to some point, and then to shipyard)
    mining = max.(0, M .- threshold)
    leftovers = min.(mining, threshold)
    ignoreshipyard = M[shipyard...]
    cost1, direction1 = travelcost(M, ship)
    cost2, direction2 = travelcost(M, shipyard)
    
    cost = cost1 + cost2 + 0.1*leftovers .- 0.1*ignoreshipyard
    direction = direction1
    
    s = mining - cost
    return s, direction
end

function filterscores(S, cost_here2there, ship_available_halite)
    #score of unreachable are zero
    S = S.*(cost_here2there .< ship_available_halite)
    return S
end

filterscores (generic function with 1 method)

In [47]:
origin=[3,2]
m=[3 5 7; 3 4 5; 1 0 2; 3 5 7]*100

4×3 Array{Int64,2}:
 300  500  700
 300  400  500
 100    0  200
 300  500  700

In [48]:
c, dir = travelcost(m, origin)
c

4×3 Array{Int64,2}:
 40  40  70
 10   0  20
  0   0   0
 10   0  20

In [49]:
dir

4×3 Array{Int64,2}:
 3  2  1
 3  2  2
 3  4  1
 3  0  1

In [50]:
ship=[3,2]
shipyard=[3,2]
threshold=9
s, dir = score(m, ship, shipyard, threshold)
s

4×3 Array{Float64,2}:
 210.1  410.1  550.1
 270.1  390.1  450.1
  90.1    0.0  190.1
 270.1  490.1  650.1

In [51]:
dir

4×3 Array{Int64,2}:
 3  2  1
 3  2  2
 3  4  1
 3  0  1

In [52]:
ship_available_halite=40
s_reachable=filterscores(s, c, ship_available_halite)

4×3 Array{Float64,2}:
   0.0    0.0    0.0
 270.1  390.1  450.1
  90.1    0.0  190.1
 270.1  490.1  650.1

In [53]:
v,i = findmax(s_reachable)
i

CartesianIndex(4, 3)

In [54]:
dir

4×3 Array{Int64,2}:
 3  2  1
 3  2  2
 3  4  1
 3  0  1

In [55]:
dir[i]



1

In [56]:
manhattandist(a,b)= sum(b-a)

manhattandist (generic function with 1 method)

In [58]:
a=[4,3]
b=[5,5]
manhattandist(a,b)

3